In [ ]:
DATA="data-dns-dsr6059"
OUTFILESUFFIX="intraffic"

import pandas as pd
import os
import  multiprocessing as mp


def filter(df):
    return df[df['src_port']=="53"]


def worker(f, ii):
    df = pd.read_csv(f,dtype=str);
    df = filter(df)
    import pyasn
    import re
    DEFAULT_ASNDB='../data/ipasn.20190116.1600.dat'
    asndb = pyasn.pyasn(DEFAULT_ASNDB)
    # src_asn = asndb.lookup(columns[1])[0]
    stat={}
    for i,dd in df.iterrows():
        if not re.match("^(?:[0-9]{1,3}\.){3}[0-9]{1,3}$", str(dd.dst)):
            print("not ipv4")
            print(str(dd.dst))
            continue
        #print(dd.dst)
        asl = asndb.lookup(str(dd.dst))[0]
        if not asl:
            # print("not aslookup")
            continue
        stat.setdefault(asl,{})
        server = str(dd.src)
        stat[asl][server] = stat[asl].get(server,0)+int(dd.iplen)
    return stat
    
p = mp.Pool(processes=4)

ii=0
results_list = []
for root, dirs, files in os.walk(DATA+"/"):
    print(root)
    print(dirs)
    print(files)
    for f in files:
        if f[-4:]==".csv":
            result = p.apply_async(worker,(root+f,ii))
            results_list.append(result)
            ii+=1

p.close()
p.join()
stat = {}
for result in results_list:
    part = result.get()
    for k,v in part.items():
        stat.setdefault(k,{})
        for ip,vol in v.items():
            stat[k][ip] = stat[k].get(ip,0)+vol


In [ ]:
import json
with open('result/dnsstat-%s-%s.json'%(DATA,OUTFILESUFFIX),'w') as f:
    json.dump(stat, f)

In [ ]:
with open('../data/stubs.txt') as f:
    t =str(f.read())
    stubs = t.split()

stat2 = {k:v for k,v in stat.items() if str(k) in stubs}
t=[(k,sum(v.values())) for k,v in stat2.items()]
t.sort(key=lambda x:x[1],reverse=True)
total = t[0:50]
stat2 = [(i[0],stat2[i[0]]) for i in t[0:50]]

In [ ]:
from matplotlib import pyplot as plt

def plot1(total):
    xlabel = []
    y=[]
    for k,v in total:
        xlabel.append(k)
        y.append(v)
    print(xlabel)
    x=list(range(len(total)))
    plt.bar(x,y)
    plt.xlabel("AS")
    plt.ylabel("total DNS traffic")
    plt.savefig("result/totalDNStraffic-%s-%s.png"%(DATA,OUTFILESUFFIX),dpi=600,bbox_inches='tight')
plot1(total)

In [ ]:

def plot2(stat2):
    xlabel=[]
    y=[]
    for k,v in stat2:
        xlabel.append(str(k))
        yy=list(v.values())
        y.append(yy)

    ml = max(len(yi) for yi in y)

    for yi in y:
        if len(yi)<ml:
            yi.extend([0 for _ in range(ml-len(yi))])
        yi.sort(reverse = True)

    x=list(range(len(xlabel)))

    ry=[]
    for yy in y:
        ry.append(list(sum(yy[i:])/float(sum(yy)) for i in range(50)))


    for i in range(min(ml,50)):
        py = [ry[ii][i] for ii in x]
        plt.bar(x, py)
    # plt.show()
    plt.xlabel('AS')
    plt.ylabel('top 50 DNS server traffic volume percent\n(each block is the percent of traffic volume \nfrom DNS server to the AS, \nthe bottom block is sum of other cases)')
    plt.savefig("result/dnsresult-%s-%s.png"%(DATA,OUTFILESUFFIX),dpi=600, bbox_inches='tight')
    return xlabel

asnum = plot2(stat2)